In [ ]:
import pandas as pd

from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist
from tqdm import tqdm
from normdata.utils import import_from_normdata

In [ ]:
df = gsheet_to_df("1HwiAHSiBDb2eNfm56Ie9GTNurBIStoqnUxFwszChDmU")
works_df = gsheet_to_df("1aygcH8NTUuDVGSJVIVj1fyI4ATPv0x1Bt8vSLb76z3k")
len(df), len(works_df)

In [ ]:
unique_authors = works_df['authorId'].unique().tolist()
len(unique_authors)
type(unique_authors[10])

In [ ]:
col, _ = Collection.objects.get_or_create(name="Die Schaubühne")
domain = "schaubuehne"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = """
„Die Schaubühne. Herausgeber: Siegfried Jacobsohn, Berlin 1905–1918“. Digitale Edition. Hrsg. v. Imelda Rohrbacher. Wien: Österreichische Akademie der Wissenschaften (ÖAW) 2024, <a href="https://schaubuehne.oeaw.ac.at">https://schaubuehne.oeaw.ac.at</a>
"""
col.collection_type = col_type
col.save()

In [ ]:
broken_wikidata = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    if isinstance(row["wikidataId"], str):
        item_id = row["id"]
        if float(item_id) in unique_authors:
            norm_uri = f'http://www.wikidata.org/entity/{row["wikidataId"]}'
            domain_uri = f"https://schaubuehne.oeaw.ac.at/register.html?author={item_id}"
            entity = import_from_normdata(norm_uri, "person")
            if entity:
                pmb_uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
                pmb_uri.entity = entity
                pmb_uri.save()
                entity.collection.add(col)
            else:
                broken_wikidata.append(norm_uri)